In [1]:
import requests
import re
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
response = requests.get(url)

if response.status_code == 200:
    with open("data/the-verdict.txt", "w") as file:
        file.write(response.text)
        print("File downloaded successfully.")
else:
    print("Failed to download the file.")


File downloaded successfully.


In [3]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    print("Total number of character:", len(raw_text))
    print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Tokenize
Naive approach based on regex

In [4]:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4649


In [5]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### Create vocabulary

In [6]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

1159


In [7]:
vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)
('Her', 51)


In [8]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        # Store the vocabulary as a class attribute for access in the encode and decode methods
        self.str_to_int = vocab
        # Create an inverse vocabulary that maps token IDs back to the original text tokens
        self.int_to_str = {i:s for s,i in vocab.items()}

    # Process input text into token IDs
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if
        item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    # Convert token IDs back into text
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [9]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn said with
pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]


In [10]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

#### Handle unknown words

In [11]:
all_words.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_words)}
print(len(vocab.items()))

1161


In [12]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Replace unknown words by <|unk|> tokens
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [13]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [14]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1160, 5, 362, 1155, 642, 1000, 10, 1159, 57, 1013, 981, 1009, 738, 1013, 1160, 7]


In [15]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


### 

## Byte Pair Encoding (Tokenizer)

In [16]:
tokenizer = tiktoken.get_encoding("gpt2")

In [17]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [18]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [19]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [20]:
enc_sample = enc_text[50:]

### Create Next-Word Prediction Data

In [21]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y: {y}")

x: [290, 4920, 2241, 287]
y: [4920, 2241, 287, 257]


In [22]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [23]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        # Tokenize the entire text
        token_ids = tokenizer.encode(txt)
        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    # Return the total number of rows in the dataset
    def __len__(self):
        return len(self.input_ids)
    
    # Return a single row from the dataset
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [24]:
def create_dataloader(txt, batch_size=4, max_length=256, stride=128):
    tokenizer = tiktoken.get_encoding("gpt2") #A
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) #B
    dataloader = DataLoader(dataset, batch_size=batch_size) #C
    return dataloader

In [25]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader(raw_text, batch_size=1, max_length=4, stride=1)
data_iter = iter(dataloader) #A
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [26]:
dataloader = create_dataloader(raw_text, batch_size=8, max_length=4, stride=5)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 3619,   402,   271, 10899],
        [  257,  7026, 15632,   438],
        [  257,   922,  5891,  1576],
        [  568,   340,   373,   645],
        [ 5975,   284,   502,   284],
        [  326,    11,   287,   262],
        [  286,   465, 13476,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [  402,   271, 10899,  2138],
        [ 7026, 15632,   438,  2016],
        [  922,  5891,  1576,   438],
        [  340,   373,   645,  1049],
        [  284,   502,   284,  3285],
        [   11,   287,   262,  6001],
        [  465, 13476,    11,   339]])


In [27]:
input_ids = torch.tensor([5, 1, 3, 2])
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [28]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [29]:
print(embedding_layer(input_ids))

tensor([[-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010],
        [-0.4015,  0.9666, -1.1481],
        [ 1.2753, -0.2010, -0.1606]], grad_fn=<EmbeddingBackward0>)


## Create Full Training Data

In [30]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [31]:
max_length = 4
dataloader = create_dataloader(raw_text, batch_size=8, max_length=max_length, stride=5)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 3619,   402,   271, 10899],
        [  257,  7026, 15632,   438],
        [  257,   922,  5891,  1576],
        [  568,   340,   373,   645],
        [ 5975,   284,   502,   284],
        [  326,    11,   287,   262],
        [  286,   465, 13476,    11]])

Inputs shape:
 torch.Size([8, 4])


In [32]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [33]:
token_embeddings[0]

tensor([[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
        [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
        [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
        [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],
       grad_fn=<SelectBackward0>)

In [34]:
block_size = max_length
pos_embedding_layer = torch.nn.Embedding(block_size, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(block_size))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [35]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [36]:
input_embeddings[0]

tensor([[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,  0.8900],
        [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493, -1.6473],
        [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,  0.8051],
        [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,  0.8310]],
       grad_fn=<SelectBackward0>)

## End of Textbook Section
Below code not in book

In [37]:
print(torch.cuda.is_available())

True


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

# Check if CUDA is available and set
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class GPT1(nn.Module):
    def __init__(self, vocab_size, output_dim, block_size, num_heads, num_layers):
        super().__init__()
        self.token_embedding_layer = nn.Embedding(vocab_size, output_dim)
        self.pos_embedding_layer = nn.Embedding(block_size, output_dim)
        self.transformer = nn.Transformer(output_dim, num_heads, num_layers)
        self.linear = nn.Linear(output_dim, vocab_size)

    def forward(self, inputs):
        token_embeddings = self.token_embedding_layer(inputs)
        batch_size, seq_length, output_dim = token_embeddings.shape
        pos_embeddings = self.pos_embedding_layer(torch.arange(seq_length))
        pos_embeddings = pos_embeddings.unsqueeze(0).repeat(batch_size, 1, 1)
        input_embeddings = token_embeddings + pos_embeddings
        transformer_out = self.transformer(input_embeddings, input_embeddings)
        logits = self.linear(transformer_out)
        return logits

    def train_model(self, train_dataloader, num_epochs, learning_rate, save_path):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            total_loss = 0
            for inputs, targets in train_dataloader:
                optimizer.zero_grad()
                logits = self.forward(inputs)
                loss = criterion(logits.view(-1, logits.size(-1)), targets.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

        # Save the model after training
        torch.save(self.state_dict(), save_path)
        print(f"Model saved to {save_path}")            

Using device: cuda


In [59]:
with open("data/genesis.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader(raw_text, batch_size=64, max_length=32, stride=33)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[13746,  9339,   352,  ...,  1203,   290,  7951],
        [  290, 11854,   373,  ...,    18,   197,  1870],
        [  531,    11,   564,  ...,   373,   922,    11],
        ...,
        [ 3656,    11,   290,  ...,   407, 22461,    13],
        [13746,  9339,   513,  ...,   284,   262,  2415],
        [  564,   250, 11633,  ...,    17,   197,   464]])

Targets:
 tensor([[ 9339,   352,    25,  ...,   290,  7951,    11],
        [11854,   373,   625,  ...,   197,  1870,  1793],
        [   11,   564,   250,  ...,   922,    11,   290],
        ...,
        [   11,   290,   484,  ..., 22461,    13,   198],
        [ 9339,   513,    25,  ...,   262,  2415,    11],
        [  250, 11633,  1793,  ...,   197,   464,  2415]])


In [60]:
model = GPT1(vocab_size=vocab_size, output_dim=output_dim, block_size=32, num_heads=4, num_layers=16)

In [61]:
model.train_model(dataloader, num_epochs=64, learning_rate=0.001, save_path="models/gpt1.pth")

In [51]:
# Load the trained model
# model = GPT1(vocab_size=vocab_size, output_dim=output_dim, block_size=5, num_heads=2, num_layers=2)
model.load_state_dict(torch.load("models/gpt1.pth"))

# Set the model to evaluation mode
model.eval()

GPT1(
  (token_embedding_layer): Embedding(50257, 256)
  (pos_embedding_layer): Embedding(32, 256)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-7): 8 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
 

In [52]:

# Generate text using the trained model
text = "And God said, “Let"
input_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(input_ids)
input_ids_trunc = input_ids[:32]
print(input_ids_trunc)
input_tensor = torch.tensor(input_ids_trunc).unsqueeze(0)
with torch.no_grad():
    output = model(input_tensor)
    print(output)
    predicted_token_ids = torch.argmax(output, dim=-1)
    print(predicted_token_ids)
    tokenizer = tiktoken.get_encoding("gpt2")
    predicted_text = tokenizer.decode(predicted_token_ids.squeeze().tolist())
    # Print the generated text
    print(predicted_text)


[1870, 1793, 531, 11, 564, 250, 5756]
[1870, 1793, 531, 11, 564, 250, 5756]
tensor([[[  0.3670,  -8.8320,  -9.6879,  ...,  -9.0256,  -8.7860, -10.1230],
         [  0.3670,  -8.8321,  -9.6879,  ...,  -9.0256,  -8.7860, -10.1230],
         [  0.3670,  -8.8321,  -9.6879,  ...,  -9.0256,  -8.7860, -10.1230],
         ...,
         [  0.3670,  -8.8321,  -9.6879,  ...,  -9.0256,  -8.7860, -10.1230],
         [  0.3670,  -8.8321,  -9.6879,  ...,  -9.0256,  -8.7860, -10.1230],
         [  0.3670,  -8.8320,  -9.6879,  ...,  -9.0256,  -8.7860, -10.1230]]])
tensor([[11, 11, 11, 11, 11, 11, 11]])
,,,,,,,


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
tokenizer.decode([11])

','